<h4>3. Eigenfaces-Face classification using PCA (40 classes)
<br><br>a) Use the following “face.csv” file to classify the faces of 40 different people.
<br>b) Do not use the in-built function for implementing PCA.
<br>c) Use appropriate classifier taught in class (any classification algorithm taught in class
like Bayes classifier, minimum distance classifier, and so on )
<br>d) Refer to the following link for a description of the dataset:
    <a href=https://towardsdatascience.com/eigenfaces-face-classification-in-python-7b8d2af3d3ea>LINK</a>
    
 <h4>predict the targets 

In [4]:
import pandas as pd
import numpy as np


In [5]:
df = pd.read_csv("data/face.csv")
print(df.shape)
df

(400, 4097)


,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,target
0,0.309917,0.367769,0.417355,0.442149,0.528926,0.607438,0.657025,0.677686,0.690083,0.685950,...,0.669422,0.652893,0.661157,0.475207,0.132231,0.148760,0.152893,0.161157,0.157025,0
1,0.454545,0.471074,0.512397,0.557851,0.595041,0.640496,0.681818,0.702479,0.710744,0.702479,...,0.157025,0.136364,0.148760,0.152893,0.152893,0.152893,0.152893,0.152893,0.152893,0
2,0.318182,0.400826,0.491736,0.528926,0.586777,0.657025,0.681818,0.685950,0.702479,0.698347,...,0.132231,0.181818,0.136364,0.128099,0.148760,0.144628,0.140496,0.148760,0.152893,0
3,0.198347,0.194215,0.194215,0.194215,0.190083,0.190083,0.243802,0.404959,0.483471,0.516529,...,0.636364,0.657025,0.685950,0.727273,0.743802,0.764463,0.752066,0.752066,0.739669,0
4,0.500000,0.545455,0.582645,0.623967,0.648760,0.690083,0.694215,0.714876,0.723140,0.731405,...,0.161157,0.177686,0.173554,0.177686,0.177686,0.177686,0.177686,0.173554,0.173554,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.400826,0.495868,0.570248,0.632231,0.648760,0.640496,0.661157,0.636364,0.665289,0.698347,...,0.396694,0.264463,0.099174,0.181818,0.243802,0.247934,0.161157,0.157025,0.136364,39
396,0.367769,0.367769,0.351240,0.301653,0.247934,0.247934,0.367769,0.512397,0.574380,0.628099,...,0.334711,0.289256,0.285124,0.338843,0.404959,0.458678,0.487603,0.512397,0.549587,39
397,0.500000,0.533058,0.607438,0.628099,0.657025,0.632231,0.657025,0.669422,0.673554,0.702479,...,0.148760,0.152893,0.161157,0.161157,0.173554,0.157025,0.177686,0.148760,0.190083,39
398,0.214876,0.219008,0.219008,0.223140,0.210744,0.202479,0.276859,0.400826,0.487603,0.549587,...,0.392562,0.367769,0.409091,0.479339,0.524793,0.545455,0.574380,0.590909,0.603306,39


In [6]:
total_targets = df['target'].nunique()
total_targets

40

In [7]:
X = df.drop('target',axis=1).to_numpy()
Y = df['target'].to_numpy()
Y.shape

(400,)

In [8]:
meanx = np.mean(X,axis = 0)
X_centered = X - meanx
xcov = np.cov((X_centered).T)
xcov.shape

(4096, 4096)

In [9]:
eigen_values,eigen_vectors = np.linalg.eig(xcov)

In [10]:
pairs = [(np.abs(eigen_values[i]),eigen_vectors[i]) for i in range(len(eigen_values))]
pairs = sorted(pairs,key = lambda x:x[0],reverse=True)

In [11]:
eigen_value_sum = sum(p[0] for p in  pairs)
required_eig_sum = 0.95*eigen_value_sum
cum_sum = 0
final_index = 0
for i,p in enumerate(pairs):
    cum_sum += p[0]
    final_index = i
    if(cum_sum >= required_eig_sum):
        break
print(eigen_value_sum,cum_sum,final_index)

79.11808585082717 75.19297855895623 122


In [12]:
print(eigen_value_sum,"percentage of eigen_values",cum_sum/eigen_value_sum*100,final_index)

79.11808585082717 percentage of eigen_values 95.03892536117277 122


<h4>so we will choose d' as 123 here

In [13]:
selected_vectors = eigen_vectors[:,0:final_index+1]

In [14]:
selected_vectors.shape

(4096, 123)

<br>X is n x 4096
<br>to get Y we need X.W
<br>which is n x 4096 x 4096 x 123
<br>that is Y will be n x 123

In [15]:
X_reduced = np.dot(X_centered,selected_vectors)
X_reduced.shape

(400, 123)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.1)
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_predicted = gnb.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score
print("Original class:","predicted class","result")
for o,p in zip(y_test,y_predicted):
    res = "correct"
    if(o != p):
        res = "in" + res
    print(o,"\t\t",p,"\t\t",res)


Original class: predicted class result
21 		 21 		 correct
18 		 18 		 correct
6 		 6 		 correct
1 		 3 		 incorrect
8 		 8 		 correct
21 		 21 		 correct
14 		 14 		 correct
30 		 30 		 correct
35 		 35 		 correct
10 		 10 		 correct
21 		 21 		 correct
5 		 5 		 correct
11 		 11 		 correct
11 		 11 		 correct
38 		 38 		 correct
15 		 15 		 correct
3 		 3 		 correct
26 		 26 		 correct
0 		 16 		 incorrect
8 		 8 		 correct
2 		 2 		 correct
20 		 20 		 correct
23 		 23 		 correct
7 		 7 		 correct
6 		 6 		 correct
5 		 5 		 correct
29 		 29 		 correct
37 		 37 		 correct
18 		 18 		 correct
17 		 17 		 correct
10 		 10 		 correct
24 		 24 		 correct
1 		 1 		 correct
29 		 29 		 correct
5 		 5 		 correct
6 		 6 		 correct
33 		 33 		 correct
29 		 29 		 correct
35 		 35 		 correct
24 		 24 		 correct


In [18]:
print("accuracy found is",str(accuracy_score(y_predicted,y_test)*100)+"%")


accuracy found is 95.0%
